In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\End-To-End-ML-Project-With-MLFlow'

In [66]:
import os
os.environ["MLFLOW_ENABLE_LEGACY_LOGGING_API"] = "true"
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Vegadhardik7/End-To-End-ML-Project-With-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Vegadhardik7" 
os.environ["MLFLOW_TRACKING_PASSWORD"]="0b097abf2f947a5cd50556274d27ae5b907535de"

In [71]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    test_data_path : Path
    model_path : Path
    all_params : dict
    metric_file_name : Path
    target_column : str
    mlflow_uri : str

In [72]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForest
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = Path(config.metric_file_name),
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/Vegadhardik7/End-To-End-ML-Project-With-MLFlow.mlflow"
        )

        return model_evaluation_config

In [73]:
import pandas as pd
import numpy as np  # kept for any downstream use
import joblib
import mlflow
from pathlib import Path
from urllib.parse import urlparse

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
)
from sklearn.preprocessing import label_binarize

from src.mlproject.utils.common import save_json


class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    # -----------------------------------------------------------
    # Metric calculation
    # -----------------------------------------------------------
    def eval_metrics(self, actual, pred, prob, class_labels):
        acc = accuracy_score(actual, pred)
        precision, recall, f1, _ = precision_recall_fscore_support(
            actual, pred, average="macro"
        )

        # Macro one‑vs‑rest ROC‑AUC
        actual_bin = label_binarize(actual, classes=class_labels)
        auc = roc_auc_score(actual_bin, prob, average="macro", multi_class="ovr")

        return {
            "accuracy": acc,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "roc_auc": auc,
        }

    # -----------------------------------------------------------
    # MLflow logging
    # -----------------------------------------------------------
    def log_into_mlflow(self):
        # 1. Load data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop(columns=[self.config.target_column])
        test_y = test_data[self.config.target_column]

        # 2. Configure MLflow URI
        mlflow.set_tracking_uri(self.config.mlflow_uri)

        with mlflow.start_run():
            # 3. Predictions
            pred_y = model.predict(test_x)
            prob_y = model.predict_proba(test_x)

            # 4. Metrics
            class_labels = sorted(test_y.unique())
            scores = self.eval_metrics(test_y, pred_y, prob_y, class_labels)

            # Save metrics locally
            save_json(path=self.config.metric_file_name, data=scores)

            # Log params & metrics to MLflow
            mlflow.log_params(self.config.all_params)
            for key, val in scores.items():
                mlflow.log_metric(key, val)

            # 5. Log model artifact (✔ works on DagsHub / any backend)
            model_file = "model.pkl"
            joblib.dump(model, model_file)
            mlflow.log_artifact(model_file)


In [74]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-06-28 18:04:56,672: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-28 18:04:56,683: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-28 18:04:56,686: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-28 18:04:56,687: INFO: common: created directory at: artifacts]
[2025-06-28 18:04:56,689: INFO: common: created directory at: artifacts/model_evaluation]
[2025-06-28 18:04:57,387: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


d:\End-To-End-ML-Project-With-MLFlow\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


🏃 View run classy-moose-497 at: https://dagshub.com/Vegadhardik7/End-To-End-ML-Project-With-MLFlow.mlflow/#/experiments/0/runs/c6fd9256d5684ecc88fcfcef328c88ff
🧪 View experiment at: https://dagshub.com/Vegadhardik7/End-To-End-ML-Project-With-MLFlow.mlflow/#/experiments/0
